# Questão 5


In [1]:
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from keras.datasets import cifar10
import numpy as np
from skimage.transform import resize

In [2]:
(x_train, _), (x_test, _) = cifar10.load_data()

# Reduzir o tamanho do conjunto de dados para 5000 exemplos de treinamento e 1000 exemplos de teste
x_train = x_train[:5000]
x_test = x_test[:1000]

# Redimensionar as imagens para 64x64 pixels e normalizar
x_train_resized = np.array([resize(image, (64, 64)) for image in x_train])
x_test_resized = np.array([resize(image, (64, 64)) for image in x_test])

x_train_resized = x_train_resized.astype('float32') / 255.0
x_test_resized = x_test_resized.astype('float32') / 255.0

# Converter para escala de cinza
x_train_resized = np.mean(x_train_resized, axis=-1, keepdims=True)
x_test_resized = np.mean(x_test_resized, axis=-1, keepdims=True)


In [3]:
input_img = Input(shape=(64, 64, 1))

# Encoder
x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
encoded = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(encoded)

# Decoder
x = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [4]:
autoencoder.fit(x_train_resized, x_train_resized,
                epochs=10,
                batch_size=128,
                shuffle=True,
                validation_data=(x_test_resized, x_test_resized))

Epoch 1/10
40/40 [==============================] - 34s 818ms/step - loss: 0.4405 - val_loss: 0.0346
Epoch 2/10
40/40 [==============================] - 32s 800ms/step - loss: 0.0217 - val_loss: 0.0207
Epoch 3/10
40/40 [==============================] - 32s 806ms/step - loss: 0.0198 - val_loss: 0.0192
Epoch 4/10
40/40 [==============================] - 31s 774ms/step - loss: 0.0185 - val_loss: 0.0180
Epoch 5/10
40/40 [==============================] - 31s 772ms/step - loss: 0.0174 - val_loss: 0.0172
Epoch 6/10
40/40 [==============================] - 30s 762ms/step - loss: 0.0167 - val_loss: 0.0166
Epoch 7/10
40/40 [==============================] - 32s 790ms/step - loss: 0.0162 - val_loss: 0.0162
Epoch 8/10
40/40 [==============================] - 31s 771ms/step - loss: 0.0159 - val_loss: 0.0158
Epoch 9/10
40/40 [==============================] - 31s 764ms/step - loss: 0.0153 - val_loss: 0.0151
Epoch 10/10
40/40 [==============================] - 32s 797ms/step - loss: 0.0147 - val_lo

In [5]:
def calculate_snr(original, reconstructed):
    mse = np.mean((original - reconstructed) ** 2)
    signal_power = np.mean(original ** 2)
    snr = 10 * np.log10(signal_power / mse)
    return snr

decoded_imgs = autoencoder.predict(x_test_resized)
snr = calculate_snr(x_test_resized, decoded_imgs)
print(f"SNR: {snr} dB")

32/32 [==============================] - 3s 71ms/step
SNR: -7.7416640520095825 dB
